<a href="https://colab.research.google.com/github/filipemcbarros/modelos_classificadores_conciliacao/blob/main/BERTikal_classificacao_conciliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas utilizadas**

In [ ]:
import pandas as pd
from google.colab import drive
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score,roc_curve,accuracy_score,matthews_corrcoef,f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!pip install unidecode
!pip install ftfy
!pip install transformers==4.2.2
!pip install pyreadr
!pip install git+https://github.com/felipemaiapolo/legalnlp
!pip install transformers torch scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 939.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Método para carregamento das localidades**

In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/base_processos/dataset_final.csv")
dataset

,Unnamed: 0,num_processo,corpus_limpo,corpus_bruto,conciliado,ano_processo,localidade
0,0,0000698-26.2019.5.08.0120,excelentissimo senhor doutor juiz direito vara...,\n\nEXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREI...,0,2020,2ª Vara do Trabalho de Ananindeua
1,1,0000748-91.2019.5.08.0010,processo acao trabalhista rito ordinario quint...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,10ª Vara do Trabalho de Belém
2,2,0000769-82.2019.5.08.0005,processo acao trabalhista rito sumarissimo tra...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,4ª Vara do Trabalho de Belém
3,3,0000786-97.2019.5.08.0109,excelentissimo senhor doutor juiz trabalho pre...,\n\n**EXCELENTÍSSIMO (A) SENHOR (A) DOUTOR (...,0,2020,1ª Vara do Trabalho de Santarém
4,4,0000792-07.2019.5.08.0109,processo acao trabalhista rito sumarissimo hai...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,1ª Vara do Trabalho de Santarém
...,...,...,...,...,...,...,...
30857,11988,0000439-56.2022.5.08.0207,peticao reclamacao trabalhist docx ozivaldo sa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ
30858,11989,0000446-48.2022.5.08.0207,jose brito nunes ltda marjoryedossantosferreir...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ
30859,11990,0000484-60.2022.5.08.0207,microsoft word inicial ppp daniel cardoso fel...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ
30860,11991,0000471-61.2022.5.08.0207,excelentissimo senhor doutor juiz vara trabalh...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ


# Funções de Limpeza para Corpus Bruto

In [ ]:
# Remover linhas vazias
dataset = dataset.dropna(subset=['corpus_limpo'])

# Contando o número de palavras na coluna 'corpus_limpo'
dataset['corpus_limpo'] = dataset['corpus_limpo'].fillna('')
dataset['num_palavras_corpus'] = dataset['corpus_limpo'].str.split().apply(len)

# Limpeza das linhas com corpus que possuem menos que 30 palavras para eliminar textos de referências a anexos e sem conteúdo útil
dataset = dataset.drop(dataset[dataset['num_palavras_corpus'] < 30].index)
dataset

<ipython-input-5-6886ad078ad0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['corpus_limpo'] = dataset['corpus_limpo'].fillna('')
<ipython-input-5-6886ad078ad0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_palavras_corpus'] = dataset['corpus_limpo'].str.split().apply(len)


,Unnamed: 0,num_processo,corpus_limpo,corpus_bruto,conciliado,ano_processo,localidade,num_palavras_corpus
0,0,0000698-26.2019.5.08.0120,excelentissimo senhor doutor juiz direito vara...,\n\nEXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREI...,0,2020,2ª Vara do Trabalho de Ananindeua,2362
1,1,0000748-91.2019.5.08.0010,processo acao trabalhista rito ordinario quint...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,10ª Vara do Trabalho de Belém,1824
2,2,0000769-82.2019.5.08.0005,processo acao trabalhista rito sumarissimo tra...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,4ª Vara do Trabalho de Belém,3089
3,3,0000786-97.2019.5.08.0109,excelentissimo senhor doutor juiz trabalho pre...,\n\n**EXCELENTÍSSIMO (A) SENHOR (A) DOUTOR (...,0,2020,1ª Vara do Trabalho de Santarém,1303
4,4,0000792-07.2019.5.08.0109,processo acao trabalhista rito sumarissimo hai...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2020,1ª Vara do Trabalho de Santarém,1600
...,...,...,...,...,...,...,...,...
30857,11988,0000439-56.2022.5.08.0207,peticao reclamacao trabalhist docx ozivaldo sa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1796
30858,11989,0000446-48.2022.5.08.0207,jose brito nunes ltda marjoryedossantosferreir...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,2178
30859,11990,0000484-60.2022.5.08.0207,microsoft word inicial ppp daniel cardoso fel...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1098
30860,11991,0000471-61.2022.5.08.0207,excelentissimo senhor doutor juiz vara trabalh...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1359


In [ ]:
from legalnlp.clean_functions import clean_bert

dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(lambda x:clean_bert(x))
dataset

,Unnamed: 0,num_processo,corpus_limpo,corpus_bruto,conciliado,ano_processo,localidade,num_palavras_corpus
0,0,0000698-26.2019.5.08.0120,excelentissimo senhor doutor juiz direito vara...,EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO ...,0,2020,2ª Vara do Trabalho de Ananindeua,2362
1,1,0000748-91.2019.5.08.0010,processo acao trabalhista rito ordinario quint...,PROCESSO: 0000748-91.2019.5.08.0010 - AÇÃO TR...,0,2020,10ª Vara do Trabalho de Belém,1824
2,2,0000769-82.2019.5.08.0005,processo acao trabalhista rito sumarissimo tra...,PROCESSO: 0000769-82.2019.5.08.0005 - AÇÃO TR...,0,2020,4ª Vara do Trabalho de Belém,3089
3,3,0000786-97.2019.5.08.0109,excelentissimo senhor doutor juiz trabalho pre...,**EXCELENTÍSSIMO (A) SENHOR (A) DOUTOR (A) JU...,0,2020,1ª Vara do Trabalho de Santarém,1303
4,4,0000792-07.2019.5.08.0109,processo acao trabalhista rito sumarissimo hai...,PROCESSO: 0000792-07.2019.5.08.0109 - AÇÃO TR...,0,2020,1ª Vara do Trabalho de Santarém,1600
...,...,...,...,...,...,...,...,...
30857,11988,0000439-56.2022.5.08.0207,peticao reclamacao trabalhist docx ozivaldo sa...,[PETIÇÃO] RECLAMAÇÃO TRABALHIST.docx Ozivaldo...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1796
30858,11989,0000446-48.2022.5.08.0207,jose brito nunes ltda marjoryedossantosferreir...,RT - José Brito x Nunes Ltda MarjoryedosSanto...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,2178
30859,11990,0000484-60.2022.5.08.0207,microsoft word inicial ppp daniel cardoso fel...,Microsoft Word - Inicial com PPP Daniel Cardo...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1098
30860,11991,0000471-61.2022.5.08.0207,excelentissimo senhor doutor juiz vara trabalh...,EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A)...,0,2022,8ª VARA DO TRABALHO DE MACAPÁ,1359


In [ ]:
import re

#texto caixa baixa
#remover aspas simples, aspas duplas, reticências, vírgula, underline
#remover quebras de linha por espaço em branco simples
def clean_text_extra_round(text):
    text = text.lower()
    text = re.sub('[‘’“”\…,°ºª§]', ' ', text)
    text = text.replace("'","")
    text = text.replace("_","")
    text = text.strip('\n')
    text = text.strip('\t')
    text = clean_roman_numbers(text)
    text = re.sub("\s+", ' ', text)
    text = text.replace("\\", ' ')
    return text

#substituir múltiplos espaços em branco por um único espaço em branco
def multiple_one_blank_spaces(text):
  text = re.sub(r'\s+', ' ', text)
  return text

#remove números
def is_digit(text):
  text = re.sub('\w*\d\w*', '', text)
  return text

#remove pontuações
def is_punctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  return text

#remove algarismos romanos
def clean_roman_numbers(text):
    pattern = r"\b(?=[mdclxvii])m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})([ii]x|[ii]v|v?[ii]{0,3})\b\.?"
    return re.sub(pattern, ' ', text)

In [ ]:
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(clean_text_extra_round)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(is_digit)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(is_punctuation)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(clean_roman_numbers)
dataset['corpus_bruto'] = dataset['corpus_bruto'].apply(multiple_one_blank_spaces)

In [ ]:
features = dataset['corpus_bruto']
classes = dataset['conciliado']
processos = dataset['num_processo']

conciliados = features[(dataset['conciliado'] == 1)]
nao_conciliados = features[(dataset['conciliado'] == 0)]

# **Modelo de Linguagem BERTikal**

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering

print(torch.cuda.is_available())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

False


device(type='cpu')

In [ ]:
from legalnlp.get_premodel import *
# Fazendo o download do modelo pre-treinado BERTikal e o seu tokenizador
get_premodel('bert')

True

In [ ]:
# Caso esteja usando o Google Colab, não esqueça de ligar a GPU para o ambiente de execução
import IPython
from IPython.display import Image
from IPython.display import clear_output

bert_model =  BertModel.from_pretrained('/content/BERTikal/').to(device)
bert_tokenizer = BertTokenizer.from_pretrained('/content/BERTikal/vocab.txt', do_lower_case=False)

clear_output()

# configuração do BERTikal
bert_model.config

tokenizer = bert_tokenizer
model = bert_model

In [ ]:
data_text = list(features)

In [ ]:
# Aplicando o bert_tokenizer em nosso dataset com um comprimento máximo de 512 tokens
encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")

#Agora temos nossos encoded_input em um dicionário com 3 chaves
encoded_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# Enviando os tensores para para a GPU
input_ids = encoded_inputs['input_ids'].to(device)

In [ ]:
from tqdm import tqdm

# Criando o nosso vetor de features
features = []

# Aplicando o modelo pré-treinado em cada frase e adicionando-o ao nosso vetor

for i in tqdm(range(len(data_text))):

    with torch.no_grad():

      last_hidden_states = bert_model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()

    features.append(last_hidden_states)

# Criando um numpy array com as features extraidas
features = np.array(features)

 63%|██████▎   | 18562/29475 [10:32:45<6:06:42,  2.02s/it]

In [ ]:
df_features = pd.DataFrame(features)
features_label = pd.concat([df_features, dataset['conciliado']], axis = 1)
features_label.shape

In [ ]:
print(features_label)

In [ ]:
!pip install catboost
clear_output()

# **Definindo Conjunto de Treinamento e Testes**

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features_label.drop(columns = ['conciliado']), features_label['conciliado'], random_state = 1234,test_size = 0.3)

# SVC

In [ ]:
from sklearn.svm import SVC

#parâmetros com valores decididos pela função GridSearchCV
svcGaussian = SVC(kernel='rbf', C=1000.0)
svcGaussian.fit(X_train,y_train)
target_svc_gaussian = svcGaussian.predict(X_test)

# **Avaliação do modelo SVC**

In [ ]:
print('############# Relatório de Classificação - Modelo SVM #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_svc_gaussian, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_svc_gaussian)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_svc_gaussian)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_svc_gaussian)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_svc_gaussian)))

############# Relatório de Classificação - Modelo Regressão Logística #############

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0     0.6041    0.5301    0.5647      3858
         1.0     0.6409    0.7072    0.6724      4576

    accuracy                         0.6262      8434
   macro avg     0.6225    0.6186    0.6185      8434
weighted avg     0.6241    0.6262    0.6231      8434

Acurácia: 0.6262

AUC: 0.6186

MCC:  0.2411

F1-Score:  0.6724



In [ ]:
print('#### Matriz de Confusão - Modelo SVM ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_svc_gaussian)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

#parâmetros com valores decididos pela função GridSearchCV
lr = LogisticRegression(solver='lbfgs', C=10.0)
lr.fit(X_train,y_train)
target_lr = lr.predict(X_test)

# **Avaliação do modelo Regressão Logística**



In [ ]:
print('############# Relatório de Classificação - Modelo Regressão Logística #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_lr, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_lr)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_lr)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_lr)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_lr)))

############# Relatório de Classificação - Modelo Regressão Logística #############

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0     0.6041    0.5301    0.5647      3858
         1.0     0.6409    0.7072    0.6724      4576

    accuracy                         0.6262      8434
   macro avg     0.6225    0.6186    0.6185      8434
weighted avg     0.6241    0.6262    0.6231      8434

Acurácia: 0.6262

AUC: 0.6186

MCC:  0.2411

F1-Score:  0.6724



In [ ]:
print('#### Matriz de Confusão - Modelo Regressão Logística ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_lr)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#parâmetros com valores decididos pela função GridSearchCV
dtc = DecisionTreeClassifier(criterion='“entropy”')
dtc.fit(word2vec_train_vectors, y_train)
target_dtc = dtc.predict(word2vec_test_vectors)

# **Avaliação do modelo Árvore de Decisão**



In [ ]:
print('############# Relatório de Classificação - Modelo Árvore de Decisão #############')
print()

print("Relatório de Classificação:\n", classification_report(y_test_carregado, target_dtc, digits=4))
# imprimir a acurácia do modelo
print("Acurácia: {:.4f}\n".format(accuracy_score(y_test_carregado, target_dtc)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test_carregado, target_dtc)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(y_test_carregado, target_dtc)))
print("F1-Score:  {:.4f}\n".format(f1_score(y_test_carregado, target_dtc)))

############# Relatório de Classificação - Modelo Regressão Logística #############

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0     0.6041    0.5301    0.5647      3858
         1.0     0.6409    0.7072    0.6724      4576

    accuracy                         0.6262      8434
   macro avg     0.6225    0.6186    0.6185      8434
weighted avg     0.6241    0.6262    0.6231      8434

Acurácia: 0.6262

AUC: 0.6186

MCC:  0.2411

F1-Score:  0.6724



In [ ]:
print('#### Matriz de Confusão - Modelo Árvore de Decisão ####')
print()
cf_matrix = confusion_matrix(y_test_carregado,target_dtc)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()

# XGBoost

In [ ]:
# Treinamento do modelo XGBoost
import xgboost as xgb

# Convert the data into DMatrix format, which is required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'criterion':'squared_error',
    'max_depth': 7,
    'learning_rate': 0.3,
    'n_estimators': 100
}

# Train the Boosted Trees model
bst = xgb.train(params, dtrain)

# Make predictions on the test set
target_xgb = bst.predict(dtest)

# **Avaliação do modelo XGBoost**

In [ ]:
# Definir um limite de decisão (exemplo: 0.5)
limite_decisao = 0.5

# Converter as previsões em classes binárias
previsoes_binarias = [1 if predicao > limite_decisao else 0 for predicao in target_xgb]

# Converter os rótulos reais em classes binárias
rotulos_binarios = [1 if rotulo > limite_decisao else 0 for rotulo in y_test]

print('############# Relatório de Classificação - Modelo XGBoost #############')
print()

print("Relatório de Classificação:\n", classification_report(rotulos_binarios, previsoes_binarias, digits=4))

print("Acurácia: {:.4f}\n".format(accuracy_score(rotulos_binarios, previsoes_binarias)))
# imprimir a área sob da curva
print("AUC: {:.4f}\n".format(roc_auc_score(rotulos_binarios, previsoes_binarias)))
print("MCC:  {:.4f}\n".format(matthews_corrcoef(rotulos_binarios, previsoes_binarias)))
print("F1-Score:  {:.4f}\n".format(f1_score(rotulos_binarios, previsoes_binarias)))

############# Relatório de Classificação - Modelo Regressão Logística #############

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0     0.6041    0.5301    0.5647      3858
         1.0     0.6409    0.7072    0.6724      4576

    accuracy                         0.6262      8434
   macro avg     0.6225    0.6186    0.6185      8434
weighted avg     0.6241    0.6262    0.6231      8434

Acurácia: 0.6262

AUC: 0.6186

MCC:  0.2411

F1-Score:  0.6724



In [ ]:
print('#### Matriz de Confusão - Modelo XGBoost ####')
print()
cf_matrix = confusion_matrix(y_test,previsoes_binarias)
sns.heatmap(cf_matrix, annot=True, fmt='.0f')
plt.show()